In [1]:
import io
import os
import re
import sqlite3
import zipfile
from contextlib import contextmanager
from pathlib import Path

import duckdb
import pandas as pd
from tqdm import tqdm

In [2]:
DATA_DIR = Path('..') / 'data'

NAME = 'occurrence_raw_idigbio_2021-02'

ZIP = str(DATA_DIR / 'iDigBio_snapshot_2021-02.zip')
DB = str(DATA_DIR / f'{NAME}.duckdb.db')
CSV = str(DATA_DIR / f'{NAME}.csv')

PREFIX = str(DATA_DIR / f'{NAME}_')
FIRST = f'{PREFIX}01.csv'
BACKUP = f'{PREFIX}01_header.csv'
GLOB = PREFIX + '*.csv'

CHUNK = 1_000_000
LINES = 5_000_000

Make working with the database a bit easier.

In [3]:
@contextmanager
def duckdb_connect(db=':memory:'):
    db = str(db)
    cxn = duckdb.connect(db)
    try:
        yield cxn
    finally:
        cxn.close()

Get the needed CSV file from the zip file.

In [4]:
# !unzip -p $ZIP occurrence_raw.csv > $CSV

Split the CSV file into manageable chunks that fit in memory

In [5]:
!split --additional-suffix=.csv --numeric-suffixes=1 --lines=$LINES $CSV $PREFIX

Copy the first file with the header so it can be removed later.

In [6]:
!mv $FIRST $BACKUP

Get the headers from the first file. We need to rename the column headers in the DB.

In [7]:
with open(BACKUP) as in_file:
    headers = in_file.readline()
headers = [h.strip() for h in headers.split(',')]

Remove the old header from that first file.

In [8]:
!tail -n +2 $BACKUP > $FIRST

Remove the backup file

In [9]:
!rm $BACKUP

Rename the columns

In [10]:
bad_names = """ group order references """.split()

def get_columns(headers):
    columns = {}
    used = set()

    for head in headers:
        col = head.split(':')[-1]
        col = re.sub(r'(?<![A-Z])([A-Z])', r'_\1', col).lower()
        col = re.sub(r'^_', '', col)
        if col in used:
            col = head.replace(':', '_')
            col = re.sub(r'(?<![A-Z])([A-Z])', r'_\1', col).lower()
            col = re.sub(r'^_', '', col)
        if col in bad_names:
            col += '_'
        columns[head] = col
        used.add(col)
    return columns


columns = get_columns(headers)

Create a table with the new column names.

In [11]:
columns = [f'{h} VARCHAR' for h in columns.values()]
columns = ', '.join(columns)

with duckdb_connect(DB) as cxn:
    cxn.execute(f'CREATE TABLE raw_data ({columns});')

Copy CSV data into the database.

In [12]:
paths = sorted(DATA_DIR.glob(GLOB))

for path in tqdm(paths):
    with duckdb_connect(DB) as cxn:
        cxn.execute(f"COPY raw_data FROM '{str(path)}';")

100%|██████████| 26/26 [40:35<00:00, 93.67s/it] 


Remove unneeded files.

In [18]:
for path in paths:
    path.unlink()

Path(CSV).unlink()

### This uses sqlite3 which is not appropriate for our needs

In [13]:
# def get_headers(zip_file):
#     with zipfile.ZipFile(ZIP) as zippy:
#         with zippy.open(zip_file) as in_file:
#             headers = in_file.readline()
#     return [h.decode().strip() for h in sorted(headers.split(b','))]

In [14]:
# def get_columns(headers):
#     columns = {}
#     used = set()

#     for head in headers:
#         col = head.split(':')[-1]
#         col = re.sub(r'(?<![A-Z])([A-Z])', r'_\1', col).lower()
#         col = re.sub(r'^_', '', col)
#         if col in used:
#             col = head.replace(':', '_')
#             col = re.sub(r'(?<![A-Z])([A-Z])', r'_\1', col).lower()
#             col = re.sub(r'^_', '', col)
#         columns[head] = col
#         used.add(col)
#     return columns

In [15]:
# def insert(zip_file, columns):
#     table = zip_file.split('.')[0]

#     with sqlite3.connect(DB) as cxn:
#         with zipfile.ZipFile(ZIP) as zippy:
#             with zippy.open(zip_file) as in_file:

#                 reader = pd.read_csv(
#                     in_file, dtype=str, keep_default_na=False, chunksize=CHUNK)

#                 if_exists = 'replace'

#                 for df in tqdm(reader):
#                     df = df.rename(columns=columns)

#                     df.to_sql(table, cxn, if_exists=if_exists, index=False)

#                     if_exists = 'append'

In [16]:
# def wrapper(zip_file):
#     headers = get_headers(zip_file)
#     columns = get_columns(headers)
#     insert(zip_file, columns)

In [17]:
# wrapper('occurrence_raw.csv')